In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn as sk
from string import punctuation
from collections import Counter
import collections

In [15]:
with open('rev_3.txt', 'r') as f:
    reviews = f.read()
with open('lab.txt', 'r') as f:
    labels_org = f.read()

all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')
all_text = ' '.join(reviews)
words = all_text.split()

vocabulary_size = 400

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    i=0
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, vocab_to_int, int_to_vocab = build_dataset(words,
                                                            vocabulary_size)
reviews_ints=[]
for each in reviews:
    reviews_ints.append([vocab_to_int.get(word,0) for word in each.split()])
print('REVIEW INT -------------->>\n',reviews_ints[0:10])

labels = np.array([int(_)  for _ in labels_org.split()]).astype(np.int64)+1
print(len(labels))
print('LABELS------------->>\n',labels[:10])

A=[len(x) for x in reviews_ints]
review_lens = Counter(A)
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

reviews_ints = [r[0:30] for r in reviews_ints if len(r) > 0]
print(len(reviews_ints),reviews_ints[0])
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

seq_len = 30
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
print('FEARTURES----------->\n',features[0,:])

T=[int_to_vocab.get(features[0,i],'UNK') for i in range(30)]
print('INT TO VOCAB----------------->>\n',T)
del T


def get_batches(x, y, batch_size=800):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

REVIEW INT -------------->>
 [[312, 0, 5, 11, 391, 12, 20, 46, 392, 140, 65, 0, 4, 393, 24], [5, 0, 0, 313, 82, 211, 0], [5, 0, 394, 0, 92, 0, 0, 4, 314, 6], [188, 212, 213], [5, 246, 189, 395, 5, 246, 189], [0, 58, 3, 5, 0, 11, 3, 0, 0, 0, 0, 315, 0, 0, 13, 0, 11, 0, 0], [5, 190, 0, 3, 0, 30, 0, 0, 22, 0, 214], [5, 190, 4, 0, 3, 0, 214, 22, 3, 0, 9, 0], [0, 0, 113, 55, 5, 0, 0, 93], [5, 53, 0, 0, 164, 8, 165, 0, 0, 0]]
1000
LABELS------------->>
 [2 2 2 2 2 0 2 2 2 2]
Zero-length reviews: 2
Maximum review length: 30
1000 [312, 0, 5, 11, 391, 12, 20, 46, 392, 140, 65, 0, 4, 393, 24]
Zero-length reviews: 0
Maximum review length: 30
FEARTURES----------->
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 312   0   5
  11 391  12  20  46 392 140  65   0   4 393  24]
INT TO VOCAB----------------->>
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'Which', 'UNK', 'BMW', 'is', 'right', 'for', 'you', 'We', 'test', 'them', 'a

In [16]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y=train_test_split(features,labels,test_size=0.2)


print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(800, 30) 
Test set: 		(200, 30)
label set: 		(800,) 
Test label set: 		(200,)


In [17]:
seq_len=30
embeding_size=30
n_input = embeding_size*seq_len
n_classes = 3
learning_rate = 0.01
training_iters = 200000
batch_size = 800
display_step = 10
n_words = len(vocab_to_int)

In [18]:
tf.reset_default_graph()
x = tf.placeholder(tf.int32, [None, seq_len])
y = tf.placeholder(tf.int64, [None])
keep_prob = tf.placeholder(tf.float32) 
dropout=1

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def conv_net(x, weights, biases, dropout):
    global embeding_size
    global seq_len
    x = tf.reshape(x, shape=[-1, embeding_size, seq_len, 1])
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 6])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 6, 12])),
    'wd1': tf.Variable(tf.random_normal([30*30*12, 150])),
    'out': tf.Variable(tf.random_normal([150, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([6])),
    'bc2': tf.Variable(tf.random_normal([12])),
    'bd1': tf.Variable(tf.random_normal([150])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [19]:
embedding = tf.Variable(tf.random_uniform((n_words, embeding_size), -1, 1))
embed = tf.nn.embedding_lookup(embedding, x)
pred = conv_net(embed, weights, biases, keep_prob)
soft_a=tf.nn.softmax(pred)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=pred))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(cost)
correct_pred = tf.equal(tf.argmax(soft_a, 1), y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

In [20]:
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    print('Initialized')
    for epoch in range(10):
        for io,(xx,yy) in enumerate(get_batches(train_x, train_y, 800), 1):
            X_batch  = xx
            y_batch  = yy
            _,acc_train=sess.run([training_op, accuracy],feed_dict={x: X_batch, y: y_batch,keep_prob: 1})
        acc_test = accuracy.eval(feed_dict={x: test_x, y: test_y, keep_prob: 1})
        temp=soft_a.eval(feed_dict={x: test_x, y: test_y, keep_prob: 1})
        temp0=np.argmax(temp,axis=1)
        temp1=sk.metrics.f1_score(test_y.reshape(-1),temp0,average='micro')
        print(epoch, "Train accuracy:",acc_train , "Test accuracy:", acc_test,'F1:',temp1)
        saver.save(sess, "checkpoints/sento-anal_f1_CNN_enbedin_1.ckpt")

Initialized
0 Train accuracy: 0.39 Test accuracy: 0.27 F1: 0.27
1 Train accuracy: 0.24875 Test accuracy: 0.625 F1: 0.625
2 Train accuracy: 0.63875 Test accuracy: 0.625 F1: 0.625
3 Train accuracy: 0.63875 Test accuracy: 0.625 F1: 0.625
4 Train accuracy: 0.63875 Test accuracy: 0.625 F1: 0.625
5 Train accuracy: 0.63875 Test accuracy: 0.625 F1: 0.625
6 Train accuracy: 0.64125 Test accuracy: 0.565 F1: 0.565
7 Train accuracy: 0.61625 Test accuracy: 0.185 F1: 0.185
8 Train accuracy: 0.21375 Test accuracy: 0.16 F1: 0.16
9 Train accuracy: 0.21375 Test accuracy: 0.21 F1: 0.21


In [9]:
temp0-test_y

array([-1, -1, -1,  1, -1,  1,  1,  0, -1,  0,  0, -1,  1, -1, -1,  1,  0,
        0, -1, -1, -1,  0, -1,  0, -1, -1,  2,  0,  0,  1,  0,  0,  0,  0,
        0,  1, -1,  0, -1,  0, -1, -1, -1,  1,  1, -1, -1,  0, -1, -1,  0,
        0,  2, -1,  0,  2,  1,  0,  1, -1,  0,  1, -1,  1,  1, -1, -1,  2,
        0,  0,  0,  1,  2,  0, -1,  0,  0,  0, -1,  0,  0, -1, -1,  0,  1,
       -1,  0,  1, -1,  0, -1,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,
       -1,  0, -1,  0,  0,  0,  0, -1,  0,  1, -1,  1, -1,  0,  0, -1,  0,
        0,  0, -1,  0,  1,  0, -1,  0,  1,  0, -1,  0, -1, -1, -1,  1, -1,
       -1,  1, -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,  2, -1, -1,  0, -1,
        0, -1, -1,  1, -1,  1,  0,  0, -1,  2,  0, -1, -1, -1,  0,  1,  0,
       -1, -1, -1,  0,  0,  1, -1,  1, -1, -1, -1, -1, -1,  0, -1, -1,  0,
       -1,  0, -1,  2, -1, -1,  0, -1, -1,  1, -1,  0, -1])